# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.876      0.374      0.690        340       5716
author                    0.763      0.532      0.702        509       5885
capital                   0.596      0.295      0.495         95       5471
contains                  0.798      0.594      0.747       3904       9280
film_performance          0.777      0.563      0.722        766       6142
founders                  0.791      0.397      0.660        380       5756
genre                     0.588      0.176      0.401        170       5546
has_sibling               0.867      0.234      0.563        499       5875
has_spouse                0.870      0.327      0.653        594       5970
is_a                      0.679      0.217      0.477        497       5873
nationality               0.600      0.189      0.419        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.555 Taluks
     2.533 Córdoba
     2.496 Valais
     ..... .....
    -1.397 Lancashire
    -1.423 he
    -1.446 who

Highest and lowest feature weights for relation author:

     2.966 books
     2.775 author
     2.392 by
     ..... .....
    -2.868 Daisy
    -3.534 infamous
    -3.546 17th

Highest and lowest feature weights for relation capital:

     3.529 capital
     1.905 city
     1.620 posted
     ..... .....
    -1.097 Roman
    -1.179 and
    -1.778 includes

Highest and lowest feature weights for relation contains:

     2.313 bordered
     2.057 Turks
     2.017 third-largest
     ..... .....
    -2.688 Mile
    -3.169 Lancashire
    -3.814 Ceylon

Highest and lowest feature weights for relation film_performance:

     4.157 starring
     3.528 opposite
     3.435 co-starring
     ..... .....
    -1.863 Gandolfini
    -2.008 fully
    -2.025 compatible

Highest and lowest feature weights for relation founders

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [12]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [13]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [14]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))                
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [17]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.831      0.462      0.716        340       5716
author                    0.839      0.440      0.710        509       5885
capital                   0.594      0.200      0.426         95       5471
contains                  0.656      0.402      0.582       3904       9280
film_performance          0.806      0.326      0.623        766       6142
founders                  0.798      0.229      0.533        380       5756
genre                     0.412      0.041      0.147        170       5546
has_sibling               0.866      0.246      0.576        499       5875
has_spouse                0.879      0.343      0.670        594       5970
is_a                      0.740      0.149      0.412        497       5873
nationality               0.750      0.179      0.458        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [15]:
def run_svm_model_factory():
    
    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    model_factory = lambda: SVC(kernel='linear')
    print(splits)
    results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)
    return results




In [16]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [64]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

{'tiny': Corpus with 3,474 examples; KB with 445 triples, 'train': Corpus with 263,285 examples; KB with 36,191 triples, 'dev': Corpus with 64,937 examples; KB with 9,248 triples, 'all': Corpus with 331,696 examples; KB with 45,884 triples}
relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.722      0.359      0.600        340       5716
author                    0.745      0.613      0.714        509       5885
capital                   0.667      0.295      0.532         95       5471
contains                  0.785      0.602      0.740       3904       9280
film_performance          0.760      0.625      0.729        766       6142
founders                  0.730      0.434      0.643        380       5756
genre                     0.595      0.294      0.494        170       5546
has_sibling               0.815      0.238      0.549        499       5875

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [14]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    #print(feature_counter)
    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bag_of_words_featurizer],
    model_factory=lambda: LogisticRegression(fit_intercept=True, solver='liblinear'),
    verbose=True)


print("done")
vectorizer = results['vectorizer']
feature_names = vectorizer.get_feature_names()
feature_counts = len(feature_names)
print(feature_counts)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.836      0.406      0.690        340       5716
author                    0.837      0.583      0.770        509       5885
capital                   0.639      0.242      0.481         95       5471
contains                  0.814      0.663      0.779       3904       9280
film_performance          0.834      0.658      0.792        766       6142
founders                  0.813      0.413      0.681        380       5756
genre                     0.714      0.265      0.533        170       5546
has_sibling               0.851      0.253      0.577        499       5875
has_spouse                0.878      0.350      0.674        594       5970
is_a                      0.739      0.233      0.516        497       5873
nationality               0.667      0.213      0.467        301       5677
parents     

In [17]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [18]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

NameError: name 'directional_bag_of_words_featurizer' is not defined

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [19]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    #print(kbt)
    
    #print(kbt.sbj+" "+ kbt.obj)
    s = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(ex.middle_POS)
        s.append(ex.middle_POS)
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        #print(ex.middle_POS)
        s.append(ex.middle_POS)
    #print(kbt.middle())
    #s = "The/DT dog/N napped/V"
    if len(s)==0:
        return feature_counter
    for item in s:
        tag_bigrams = get_tag_bigrams(item)
        for pair in tag_bigrams:
            feature_counter[pair] +=1

    return feature_counter

def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    
    tags = get_tags(s)
    tags.insert(0, start_symbol)
    tags.append(end_symbol)
    tag_bigrams = []
    for i in range(len(tags)-1):
        pair = tags[i]+" "+tags[i+1]
        tag_bigrams.append(pair)
        
    return tag_bigrams
        
    


    
def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_bigram_pos_tag_featurizer],
    model_factory=lambda: LogisticRegression(fit_intercept=True, solver='liblinear'),
    verbose=True)
print("done")



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.914      0.344      0.687        340       5716
author                    0.728      0.332      0.588        509       5885
capital                   0.467      0.147      0.326         95       5471
contains                  0.758      0.586      0.716       3904       9280
film_performance          0.707      0.441      0.631        766       6142
founders                  0.536      0.158      0.362        380       5756
genre                     0.569      0.171      0.388        170       5546
has_sibling               0.639      0.170      0.412        499       5875
has_spouse                0.757      0.263      0.550        594       5970
is_a                      0.561      0.167      0.381        497       5873
nationality               0.386      0.073      0.208        301       5677
parents     

In [20]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [20]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [21]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    s = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(ex.middle_POS)
        s.append(ex.middle_POS)
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        #print(ex.middle_POS)
        s.append(ex.middle_POS)
    if len(s)==0:
        return feature_counter
    
    for item in s:
        res = get_synsets(item)
        for pair in res:
            feature_counter[pair] +=1
    
    #print(feature_counter)    
    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    #print(s)
    #print(wt)
    synsets = []
    for pair in wt:
        pos = convert_tag(pair[1])
        synlist = wn.synsets(pair[0], pos=pos)
        for item in synlist:
            res = str(item)
            synsets.append(res)
            #print(res)
    return synsets 
    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None    


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer],
    model_factory=lambda: LogisticRegression(fit_intercept=True, solver='liblinear'),
    verbose=True)
print("done")



//anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.804      0.338      0.630        340       5716
author                    0.748      0.438      0.655        509       5885
capital                   0.571      0.295      0.481         95       5471
contains                  0.767      0.582      0.721       3904       9280
film_performance          0.790      0.559      0.729        766       6142
founders                  0.711      0.382      0.606        380       5756
genre                     0.508      0.176      0.369        170       5546
has_sibling               0.825      0.226      0.540        499       5875
has_spouse                0.904      0.300      0.644        594       5970
is_a                      0.616      0.213      0.447        497       5873
nationality               0.454      0.146      0.319        301       5677
parents     

In [21]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():   
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [23]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [28]:
# Enter your system description in this cell.
# Please do not remove this comment.
"""
Some observations about f-score:
baseline bow featurizer = 0.558 
GloVe = 0.515 
svm = 0.560
directional_uni = 0.605
middle_bigram_pos_tag_featurizer = 0.434 
synset_featurizer = 0.526

I ran experiment with directional glove logisitc:
Glove_directional=0.547
From directional_bag_of_words & glove directional with logistic
Seems that directional features improved performace.
This makes sense because not all relations are symetric
I tried multiple attempts with glove-SVM directional but it did not suceed and my system crashed
So I evolved my system with gloves to a directional_glove with (sum,min,max) features concataned
relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.571      0.447      0.541        340       5716
author                    0.743      0.660      0.725        509       5885
capital                   0.341      0.326      0.338         95       5471
contains                  0.766      0.726      0.758       3904       9280
film_performance          0.740      0.685      0.729        766       6142
founders                  0.519      0.503      0.516        380       5756
genre                     0.496      0.341      0.455        170       5546
has_sibling               0.540      0.335      0.481        499       5875
has_spouse                0.605      0.448      0.565        594       5970
is_a                      0.486      0.318      0.440        497       5873
nationality               0.372      0.339      0.365        301       5677
parents                   0.612      0.542      0.597        312       5688
place_of_birth            0.322      0.330      0.324        233       5609
place_of_death            0.188      0.245      0.198        159       5535
profession                0.335      0.263      0.318        247       5623
worked_at                 0.405      0.360      0.395        242       5618
------------------    ---------  ---------  ---------  ---------  ---------
macro-average             0.503      0.429      0.484       9248      95264

The score went down. 

"""


## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [23]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    def glove_bi_di_middle_featurizer(kbt, corpus, np_func=[np.sum, np.min, np.max]):
        final_reps = []
        for func in np_func:
            #reps_so = []
            #reps_os = []
            so_feature = glove_middle_featurizer(kbt, corpus, func)
            final_reps.append(so_feature)
            if directional:
                kbt_os = rel_ext.KBTriple(rel=kbt.rel, sbj=kbt.obj, obj=kbt.sbj)
                os_feature = glove_middle_featurizer(kbt_os, corpus, func)

                #final_reps.append(np.concatenate((so_feature, os_feature), axis=None))
                final_reps.append( os_feature) 
            
        final_rep = np.concatenate(final_reps, axis=None)
        return final_rep
        
    """
        for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
            for word in ex.middle.split():
                rep = glove_lookup.get(word)
                if rep is not None:
                    reps_so.append(rep)
        
        for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
            for word in ex.middle.split():
                rep = glove_lookup.get(word)
                if rep is not None:
                    reps_os.append(rep)
                    #reps_so.append(rep)

        # A random representation of the right dimensionality if the
        # example happens not to overlap with GloVe's vocabulary:
        reps = [reps_so, reps_os]
        #print(len(reps_so), len(reps_os))
        summed_glove = []
        for rep in reps:
            if len(rep) == 0:
                dim = len(next(iter(glove_lookup.values())))                
                curr_rep = utils.randvec(n=dim)
            else:
                curr_rep = np_func(reps_so, axis=0)
            summed_glove.append(curr_rep)
            
        #final_rep = np.concat
        
        #print(len(reps_so, len(reps_os))
        try:
            
            print("summed_glove[0]", len(summed_glove[0]))
            print("summed_glove[1]", len(summed_glove[1]))
            final_rep = np.concatenate((summed_glove[0], summed_glove[1]), axis=None)
        except:
            print("An exception occurred")
            import pdb;
            pdb.set_trace()
        
            
        #print(final_rep.shape)
        #return final_rep
        
        for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
            for word in ex.middle.split(' '):
                feature_counter[word+subject_object_suffix] += 1
        for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
            for word in ex.middle.split(' '):
                feature_counter[word+object_subject_suffix] += 1
        """
    print("Executing experiment")    
    
    model_factory_logistic = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
    model_factory_svc = lambda: SVC(kernel='linear')
    #factories = [model_factory_logistic, model_factory_svc]
    factories = [model_factory_logistic]
    for model_factory in factories:
        results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[glove_bi_di_middle_featurizer],
        model_factory=model_factory,
        vectorize=False,
        verbose=True)
    print("Done!")    





Executing experiment


//anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
//anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
//anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
//anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.571      0.447      0.541        340       5716
author                    0.743      0.660      0.725        509       5885
capital                   0.341      0.326      0.338         95       5471
contains                  0.766      0.726      0.758       3904       9280
film_performance          0.740      0.685      0.729        766       6142
founders                  0.519      0.503      0.516        380       5756
genre                     0.496      0.341      0.455        170       5546
has_sibling               0.540      0.335      0.481        499       5875
has_spouse                0.605      0.448      0.565        594       5970
is_a                      0.486      0.318      0.440        497       5873
nationality               0.372      0.339      0.365        301       5677
parents     

In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE


